In [19]:
import os, tarfile
import re, glob
import pandas as pd
import numpy as np
import zipfile, gzip
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [20]:
df_train = pd.read_csv("/data/private/pdutta/DNABERT_data/prom-300/prom-300/tata_test.tsv", sep="\t")

In [21]:
df_train

,setence,label
0,CTCTCCCATTCTTTCCTTCATAGCATGACCTCAGGCTTGCCTTTTG...,1
1,CAAACTTACTAAAGACCGGTCCTGATTAGAGGGAAAAAATGCACCC...,1
2,GTACTTGTTCAGTATTGTCATCAGAAGCATTCACTCATTTTTAATT...,0
3,TTCGGGATTAGGGTCTCTCCCACAGGGTTTGGGGTTCTCTCTGCAG...,1
4,GCGAAGGGCAAGCGCCGGATCCTTTCGCCCCCTCCCCACGTGAGGC...,1
...,...,...
607,AGAAGGAAACCTCCCAGGAGATGAGAGATCAGGCAGACAAACAGGG...,1
608,TCTTCCTCATTTTCTCTTGCTGCTGCCATGTAAGAAGAACTTTTCA...,0
609,AGGGAGCGCAGCCTGAGGCGGTGTGGTGCGGTGCGGCTGCAGGGCG...,1
610,CCTGACCAAAGAATCCAGAGTCAAGACCTCTCCGTGCTCCATTTAG...,1


In [22]:
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [23]:
def core_prom_35to34(text):
    return text[214:284]

def core_prom_45to44(text):
    return text[204:294]

In [24]:
df_train['CoreProm-35to34']= df_train['setence'].apply(core_prom_35to34)
df_train['CoreProm-45to44']= df_train['setence'].apply(core_prom_45to44)

In [25]:
df_train

,setence,label,CoreProm-35to34,CoreProm-45to44
0,CTCTCCCATTCTTTCCTTCATAGCATGACCTCAGGCTTGCCTTTTG...,1,GGAATAAAAATATCACCGTCCCTGCCCTTGGATTCACATTGCATTC...,CGAGTGACAAGGAATAAAAATATCACCGTCCCTGCCCTTGGATTCA...
1,CAAACTTACTAAAGACCGGTCCTGATTAGAGGGAAAAAATGCACCC...,1,AACATTATAAACAGAGGAATGTAAGCTGTAAAACCAGTCCAAGAAG...,CAGCCCCCACAACATTATAAACAGAGGAATGTAAGCTGTAAAACCA...
2,GTACTTGTTCAGTATTGTCATCAGAAGCATTCACTCATTTTTAATT...,0,GAGCTATAAAATGCAAATTTTAAAAGCTTCATATTTTCATCCTGGG...,AAGAAAGGTAGAGCTATAAAATGCAAATTTTAAAAGCTTCATATTT...
3,TTCGGGATTAGGGTCTCTCCCACAGGGTTTGGGGTTCTCTCTGCAG...,1,CCGCATATATAGCAGCGGCGGCGGTGGCGGCGGCCACACCGGGCGG...,CCCGGGTGCGCCGCATATATAGCAGCGGCGGCGGTGGCGGCGGCCA...
4,GCGAAGGGCAAGCGCCGGATCCTTTCGCCCCCTCCCCACGTGAGGC...,1,GTAAGAGATAAAGGCAAGGGCGCCTGACCGGGCCTGGGCACCTCCT...,GGGAGCGAGAGTAAGAGATAAAGGCAAGGGCGCCTGACCGGGCCTG...
...,...,...,...,...
607,AGAAGGAAACCTCCCAGGAGATGAGAGATCAGGCAGACAAACAGGG...,1,AAATAAATATAAGAGGTCTGCCCTACAGTGAGGGCAGGATAGCTGA...,AGGGGCTCAGAAATAAATATAAGAGGTCTGCCCTACAGTGAGGGCA...
608,TCTTCCTCATTTTCTCTTGCTGCTGCCATGTAAGAAGAACTTTTCA...,0,GTTGTTTATAAAGAAAAGAGGTTTCTTTGGCTCACGGTTCTGCAGG...,CTGAAGCTGGGTTGTTTATAAAGAAAAGAGGTTTCTTTGGCTCACG...
609,AGGGAGCGCAGCCTGAGGCGGTGTGGTGCGGTGCGGCTGCAGGGCG...,1,GAACTTTAAAAAGCTGCTTCGGACAAACCAGAGCCAGGATTTCCAC...,GGTGCCAAGCGAACTTTAAAAAGCTGCTTCGGACAAACCAGAGCCA...
610,CCTGACCAAAGAATCCAGAGTCAAGACCTCTCCGTGCTCCATTTAG...,1,CACCTCTATATATAGCCCCTCGAAGACAGCTGCTCAGTCTAAGCAA...,GGGACCCGGCCACCTCTATATATAGCCCCTCGAAGACAGCTGCTCA...


In [26]:
print(len(df_train['setence'][0]))
print(len(df_train['CoreProm-35to34'][0]))
print(len(df_train['CoreProm-45to44'][0]))

300
70
90


In [27]:
df_Prom_TATA= pd.DataFrame()
df_coreProm70_TATA =pd.DataFrame()
df_coreProm90_TATA =pd.DataFrame()

In [28]:
df_Prom_TATA['Sequence'] = df_train['setence'].apply(seq2kmer, args=(6,))
df_Prom_TATA['Label'] = df_train['label']
df_Prom_TATA

,Sequence,Label
0,CTCTCC TCTCCC CTCCCA TCCCAT CCCATT CCATTC CATT...,1
1,CAAACT AAACTT AACTTA ACTTAC CTTACT TTACTA TACT...,1
2,GTACTT TACTTG ACTTGT CTTGTT TTGTTC TGTTCA GTTC...,0
3,TTCGGG TCGGGA CGGGAT GGGATT GGATTA GATTAG ATTA...,1
4,GCGAAG CGAAGG GAAGGG AAGGGC AGGGCA GGGCAA GGCA...,1
...,...,...
607,AGAAGG GAAGGA AAGGAA AGGAAA GGAAAC GAAACC AAAC...,1
608,TCTTCC CTTCCT TTCCTC TCCTCA CCTCAT CTCATT TCAT...,0
609,AGGGAG GGGAGC GGAGCG GAGCGC AGCGCA GCGCAG CGCA...,1
610,CCTGAC CTGACC TGACCA GACCAA ACCAAA CCAAAG CAAA...,1


In [29]:
df_coreProm70_TATA['Sequence'] = df_train['CoreProm-35to34'].apply(seq2kmer, args=(6,))
df_coreProm70_TATA['Label'] = df_train['label']
df_coreProm70_TATA

,Sequence,Label
0,GGAATA GAATAA AATAAA ATAAAA TAAAAA AAAAAT AAAA...,1
1,AACATT ACATTA CATTAT ATTATA TTATAA TATAAA ATAA...,1
2,GAGCTA AGCTAT GCTATA CTATAA TATAAA ATAAAA TAAA...,0
3,CCGCAT CGCATA GCATAT CATATA ATATAT TATATA ATAT...,1
4,GTAAGA TAAGAG AAGAGA AGAGAT GAGATA AGATAA GATA...,1
...,...,...
607,AAATAA AATAAA ATAAAT TAAATA AAATAT AATATA ATAT...,1
608,GTTGTT TTGTTT TGTTTA GTTTAT TTTATA TTATAA TATA...,0
609,GAACTT AACTTT ACTTTA CTTTAA TTTAAA TTAAAA TAAA...,1
610,CACCTC ACCTCT CCTCTA CTCTAT TCTATA CTATAT TATA...,1


In [30]:
df_coreProm90_TATA['Sequence'] = df_train['CoreProm-45to44'].apply(seq2kmer, args=(6,))
df_coreProm90_TATA['Label'] = df_train['label']
df_coreProm90_TATA

,Sequence,Label
0,CGAGTG GAGTGA AGTGAC GTGACA TGACAA GACAAG ACAA...,1
1,CAGCCC AGCCCC GCCCCC CCCCCA CCCCAC CCCACA CCAC...,1
2,AAGAAA AGAAAG GAAAGG AAAGGT AAGGTA AGGTAG GGTA...,0
3,CCCGGG CCGGGT CGGGTG GGGTGC GGTGCG GTGCGC TGCG...,1
4,GGGAGC GGAGCG GAGCGA AGCGAG GCGAGA CGAGAG GAGA...,1
...,...,...
607,AGGGGC GGGGCT GGGCTC GGCTCA GCTCAG CTCAGA TCAG...,1
608,CTGAAG TGAAGC GAAGCT AAGCTG AGCTGG GCTGGG CTGG...,0
609,GGTGCC GTGCCA TGCCAA GCCAAG CCAAGC CAAGCG AAGC...,1
610,GGGACC GGACCC GACCCG ACCCGG CCCGGC CCGGCC CGGC...,1


In [31]:
df_Prom_TATA.to_csv("/data/private/pdutta/DNABERT_data/TATA_Prom-249to50/Evaluation_data/dev.tsv", sep="\t", index=False)
df_coreProm70_TATA.to_csv("/data/private/pdutta/DNABERT_data/TATA_CoreProm-35to34/Evaluation_data/dev.tsv", sep="\t", index=False)
df_coreProm90_TATA.to_csv("/data/private/pdutta/DNABERT_data/TATA_CoreProm-45to44/Evaluation_data/dev.tsv", sep="\t", index=False)